# pre-GWAS with Hail: Sample QC

This notebook shows how to perform sample QC on a Hail MatrixTable as a pre-GWAS step and save it as a Table to an Apollo database (dnax://) on the DNAnexus platform. See documentation for guidance on launch specs for the JupyterLab with Spark Cluster app for different data sizes: https://documentation.dnanexus.com/science/using-hail-to-analyze-genomic-data


Note: For population scale data, samples may be referred to as individuals. In this notebook, the word "sample" will be used.

Pre-conditions for running this notebook successfully:
- There is an existing Hail MatrixTable in DNAX

## 1) Initiate Spark and Hail

In [ ]:
# Running this cell will output a red-colored message- this is expected.
# The 'Welcome to Hail' message in the output will indicate that Hail is ready to use in the notebook.

from pyspark.sql import SparkSession
import hail as hl

builder = (
    SparkSession
    .builder
    .enableHiveSupport()
)
spark = builder.getOrCreate()
hl.init(sc=spark.sparkContext)

## 2) Read MT

In [ ]:
# define MT url

mt_url = "dnax://database-GFpXJ5j0vzZxPZQ2Ggf14x7q/geno.mt"

In [ ]:
# read MT

mt = hl.read_matrix_table(mt_url)

In [ ]:
# View structure of MT before QC

mt.describe()

## 3) Run Hail's sample QC method

*Additional documentation: https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc*

In [ ]:
# Run sample-level QC

qc_mt = hl.sample_qc(mt)

In [ ]:
# View structure of MT after QC

qc_mt.describe()

We can see that a new column field called 'sample_qc' has been added the MT

## 4) Create sample QC Table and save in Apollo Database

In [ ]:
# Create Hail Table from MT

qc_tb = qc_mt.cols()

In [ ]:
# Define database and table name

db_name = "database_name"
tb_name = "sample_qc.ht"

In [ ]:
# Create database in DNAX

stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
print(stmt)
spark.sql(stmt).show()

In [ ]:
# Store Table in DNAX

import dxpy

# find database ID of newly created database using a dxpy method
db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
url = f"dnax://{db_uri}/{tb_name}"

# Before this step, the Hail Table is just an object in memory. To persist it and be able to access 
# it later, the notebook needs to write it into a persistent filesystem (in this case DNAX).
# See https://hail.is/docs/0.2/hail.Table.html#hail.Table.write for additional documentation.
qc_tb.write(url) # Note: output should describe size of Table (i.e. number of rows, partitions)